# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 05**: Data pipeline with Neo4j

**Date**: October 2nd 2025

**Student Name**:Jaime Antonio Contreras Barragan

**Professor**: Pablo Camarillo Ramirez

# Dataset description

Es un dataset de Malware detection de android (https://www.kaggle.com/datasets/subhajournal/android-malware-detection) con varia informacion, siendo la principal para esta practica, IP de origen, IP de destion y Label(Que es el tipo de "software" que se detecto)
Cuenta con 355,000+ registros, cada uno representa una conexion de tipo diferente.
El objetivo sera mapear todas estas diferentes conexiones con diferente tipo entre cada ip.

# Data ingestion

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.neo4j:neo4j-connector-apache-spark_2.13:5.3.10_for_spark_3") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.neo4j#neo4j-connector-apache-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f6aea814-4353-43c8-81d6-b3ac62e23c06;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.13;5.3.10_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.13_common;5.3.10_for_spark_3 in central
	found org.neo4j#caniuse-core;1.3.0 in central
	found org.neo4j#caniuse-api;1.3.0 in central
	found org.jetbrains.kotlin#kotlin-stdlib;2.1.20 in central
	found org.jetbrains#annotations;13.0 in central
	found org.neo4j#caniuse-neo4j-detection;1.3.0 in central
	found org.neo4j.driver#neo4j-java-driver-slim;4.4.21 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found io.netty#netty-handler;4.1.

In [2]:
ls -l /opt/spark/work-dir/data/android_malware

total 201500
-rwxrwxrwx 1 root root 206333281 Oct  4 16:48 Android_Malware.csv*


In [3]:
from jaimodule.spark_utils import SparkUtils
# Build schema
"""
Aqui lo intente pero son 86 columnas, de las cuales solo ocupo 3, asi que me di la libertad de recortar este primer paso.
malware_android_schema = SparkUtils.generate_schema([("Source IP", "string"), ("Destination IP", "string"),("Label", "string")])
"""
# Import your module
base_path = "/opt/spark/work-dir/data/android_malware/"
malware_df = spark.read \
                .option("header", "true") \
                .csv(base_path) \
                .select(" Source IP"," Destination IP","Label") 

#malware_df.show(n=5)
#print(f"Number of records to be persisted to files: {malware_df.count()}")

# Transformations

In [4]:
# Add the code for your transformations to create nodes and edges DataFrames HERE
from pyspark.sql.functions import col

malware_df = malware_df.dropna()
print(f"Number of records to be persisted to files: {malware_df.count()}")

malware_df = malware_df.select(
    col(" Source IP").alias("SourceIP"),
    col(" Destination IP").alias("DestinationIP"),
    col("Label")
)
malware_df.printSchema()

#malware_df.show(n=5)
source_df = malware_df.select(col("SourceIP").alias("IP"))
destination_df = malware_df.select(col("DestinationIP").alias("IP"))

nodes_df = source_df.union(destination_df).distinct()

edges_df = malware_df.select(
    col("SourceIP").alias("src"),   # source node (User)
    col("DestinationIP").alias("dst"),   # destination node (Product)
    col("Label")
)
"""
Aqui se separan los edges para poder poner las relaciones por su nombre.
"""
Android_Adware_df = edges_df.filter(col("Label")=="Android_Adware")
#Android_Adware_df.show(n=5)
Android_Scareware_df = edges_df.filter(col("Label")=="Android_Scareware")
#Android_Scareware_df.show(n=5)
Android_Malware_df = edges_df.filter(col("Label")=="Android_SMS_Malware")
#Android_SMS_Malware_df.show(n=5)
Benign_df = edges_df.filter(col("Label")=="Benign")
#Benign_df.show(n=5)


#edges_df.show(n=5)
#nodes_df.show(n=5)


Number of records to be persisted to files: 355630
root
 |-- SourceIP: string (nullable = true)
 |-- DestinationIP: string (nullable = true)
 |-- Label: string (nullable = true)



In [5]:
nodes_df.printSchema()

root
 |-- IP: string (nullable = true)



# Writing Data in Neo4j

In [6]:
# Add the code to write a graph from PySpark's DataFrames to Neo4j

neo4j_url = "bolt://neo4j-iteso:7687"
neo4j_user = "neo4j"
neo4j_passwd = "neo4j@1234"

In [7]:


nodes_df.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("labels", ":IP") \
  .option("node.keys", "IP") \
  .save()

print(f"{nodes_df.count()} nodes wrote in Neo4j")

[Stage 7:============================================>              (3 + 1) / 4]

4839 nodes wrote in Neo4j


In [8]:
Android_Adware_df.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Append") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("relationship", "Android_Adware") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship.source.labels", "IP") \
  .option("relationship.source.save.mode", "match") \
  .option("relationship.source.node.keys", "src:IP") \
  .option("relationship.target.labels", "IP") \
  .option("relationship.target.save.mode", "match") \
  .option("relationship.target.node.keys", "dst:IP") \
.save()

print(f"{Android_Adware_df.count()} edges adware wrote in Neo4j")

[Stage 14:=============================>                            (1 + 1) / 2]

147443 edges adware wrote in Neo4j


In [9]:
Android_Scareware_df.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Append") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("relationship", "Android_Scareware") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship.source.labels", "IP") \
  .option("relationship.source.save.mode", "match") \
  .option("relationship.source.node.keys", "src:IP") \
  .option("relationship.target.labels", "IP") \
  .option("relationship.target.save.mode", "match") \
  .option("relationship.target.node.keys", "dst:IP") \
.save()

"""
Aqui el error ya no lo borre, porque tardaba como 30 min en pasarse los datos
"""


NameError: name 'Android_Scarware_df' is not defined

In [10]:
print(f"{Android_Scareware_df.count()} edges scarware wrote in Neo4j")

[Stage 18:=============================>                            (1 + 1) / 2]

117082 edges scarware wrote in Neo4j


In [11]:
Android_Malware_df.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Append") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("relationship", "Android_Malware") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship.source.labels", "IP") \
  .option("relationship.source.save.mode", "match") \
  .option("relationship.source.node.keys", "src:IP") \
  .option("relationship.target.labels", "IP") \
  .option("relationship.target.save.mode", "match") \
  .option("relationship.target.node.keys", "dst:IP") \
.save()

print(f"{Android_Malware_df.count()} edges malware wrote in Neo4j")

[Stage 22:=============================>                            (1 + 1) / 2]

67397 edges malware wrote in Neo4j


In [12]:

Benign_df.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Append") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("relationship", "Benign_df") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship.source.labels", "IP") \
  .option("relationship.source.save.mode", "match") \
  .option("relationship.source.node.keys", "src:IP") \
  .option("relationship.target.labels", "IP") \
  .option("relationship.target.save.mode", "match") \
  .option("relationship.target.node.keys", "dst:IP") \
.save()

print(f"{Benign_df.count()} edges benign wrote in Neo4j")

[Stage 26:=============================>                            (1 + 1) / 2]

23708 edges benign wrote in Neo4j


# Read and Query Graphs with PySpark

In [19]:
# Add the code to read a data frame from Neo4J and run a simple query to verify 

"""
Regresamos las IP´s unicas a las que se relaciona con Malware siendo 10.42.0.151 el nodo origen
"""
cypher_df = spark.read \
    .format("org.neo4j.spark.DataSource") \
    .option("url", neo4j_url) \
    .option("authentication.basic.username", neo4j_user) \
    .option("authentication.basic.password", neo4j_passwd) \
    .option("query",
            """
            MATCH (a)-[r:Android_Malware]->(b)
            WHERE a.IP = '10.42.0.151' 
            RETURN DISTINCT b.IP
            """) \
    .load()
cypher_df.show(n=5)

[Stage 32:>                                                         (0 + 1) / 1]

+---------------+
|           b.IP|
+---------------+
| 202.77.129.185|
|192.229.163.180|
|  119.146.74.33|
|192.229.173.136|
|  182.254.5.234|
+---------------+
only showing top 5 rows


In [13]:
sc.stop()